In [1]:
import phytomap_registration 

import io
import numpy as np
from bigstream import transform
from matplotlib import pyplot as plt
import zarr
from bigstream import affine


from skimage import io
from numcodecs import Blosc
from dask.array import to_zarr

from IPython.display import display, HTML
display(HTML("<style>.container { height:100% !important; }</style>"))
import time


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
#common parameters
in_dir = ".../FOV1/"
out_main = "global_align" #name of output directory
img_dir = "input_img"


spacing = [0.36,0.36,0.42]#[xyz] in um
slc = 150 # z plane to show for checking registered images
min_radius=6 # radius in voxels of smallest expected blob/cell size
max_radius=30 # radius in voxels of largest expected blob/cell size
match_threshold=0.75 # neighborhood correlation between two key points must exceed this value for it to be a valid match
im_rounds = ["1","2","3","4","5","6","7"] # all imaging rounds

In [ ]:

for idx in im_rounds:

    ####################
    im_round = idx # imaging round to register
    fixround = "1" # reference round
    reg_channel = "CH2" # channel for registlation, such as cell boundary staining. 
    channels = ["0","1","2","3","4"] # channels to register
    ####################

    print("registering round " + idx + "...")
    startTime = time.time()

    # select images for registration.
    # assuming the image files are named in the following way: R1_CH0.tif
    impath_fix_highres = in_dir + img_dir + "/R" + fixround +"_"+ reg_channel +".tif"
    impath_mov_highres = in_dir + img_dir + "/R" + im_round +"_"+ reg_channel +".tif"

    image_prefix = "R" + im_round + "_"
    



    # run global alignment  
    global_affine, mov_highres_aligned_global = phytomap_registration.global_affine_reg(impath_fix=impath_fix_highres, 
                                                          impath_mov=impath_mov_highres,
                                                          spacing=spacing,
                                                          downsampling=[1,1,1],
                                                          slc=slc,
                                                          min_radius=min_radius,
                                                          max_radius=max_radius,
                                                          match_threshold=match_threshold

                                                         )


   


    # apply an affine transformation to each image 
    phytomap_registration.round_submit(im_round=im_round,
                image_prefix=image_prefix,
                imdir=in_dir,
                slc=slc,
                channels=channels,
                impath_fix_highres=impath_fix_highres,
                spacing=spacing,
                global_affine=global_affine,
                out_main = out_main,
                img_dir = img_dir
    )
    
    executionTime = (time.time() - startTime)
    print('Execution time in seconds: ' + str(executionTime))